In [ ]:
from itertools import chain, combinations

: 

In [ ]:
def verify1NF():
    file = open("data.txt","r")
    fread = file.read()
    splitedData = fread.split("|")
    
    count = 0
    temp_lst = []
    for i in splitedData:
        if(i.count(",")):
            count += 1
            temp_lst.append(i)
    file.close()
    
    if (count == 0):
        return True, temp_lst
    else:
        return False, temp_lst
    file.close()

: 

In [ ]:
def MainCode():
    
    f1 = open("test.txt",'r')
    lines = []
    fread1=f1.readline()
    #print(fread1)
#     for l in sys.stdin:
#         if l.strip() != "":
#             lines.append(l.strip())

    fr1 = f1.readlines()[1:]
    st1 = ""
    st1 = st1.join(fr1)
    st1 = st1.split("\n")
    
    FDS = []
    for i in st1:
        lhs = i.split('->')[0]
        rhs = i.split('->')[1]
        lhs = set([ x.strip() for x in lhs.split(',')])
        rhs = set([ x.strip() for x in rhs.split(',')])

        FD = (lhs, rhs)
        FDS.append( FD )
    f1.close()
    
    return FDS

: 

In [ ]:
def getRelations():
    f2 = open("test.txt",'r')
    fr = f2.readline()
    data = list(fr.split("\n")[0])
    ans = data[3 : len(data)-2]
    i=0
    while (i != len(ans)-1):
        if (i % 2 != 0):
            ans.remove(',')
        i = i + 1
    f2.close()
    return set(ans)

: 

In [ ]:
def getAttributeClosure(X, FDS):
    '''
    Computes the attribute colsure given a set of attribute and functional dependencies.
    X: key attributes
    FDS: list of functional dependencies
    '''
    retval = set(X)

    breaks = False

    while not breaks:
        breaks = True
        for fd in FDS:
            lhs = fd[0]
            rhs = fd[1]

            if lhs.issubset(retval) and not rhs.issubset(retval):
                breaks = False
                retval.update(rhs)

    return retval

: 

In [ ]:
def fetchCK():
    f2 = open("test.txt","r")
    temp1 = f2.readlines()
    ck = temp1[1].split(":-")
    f2.close()
    return ck[1].split("\n")

: 

In [ ]:
def getSuperKeys(R, FDS):
    '''
    Computes the super keys
    '''
    s = list(R)
    subsets = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

    supers = []

    for subset in subsets:
        closure = getAttributeClosure(set(subset), FDS)
        if closure == R:
            supers.append( set(subset) )

    return supers

: 

In [ ]:
def getPrimeAttributes(FDS, CK):
    '''
    Computes the prime attributes, given a relation and functional dependencies
    '''
    retset = set([])

    candidates = CK
    for key in candidates:
        for a in key:
            retset.add( a )
    return retset

: 

In [ ]:
def getNonPrimeAttributes(R, FDS, CK):
    '''
    Computes the prime attributes, given a relation and functional dependencies
    '''
    primes = getPrimeAttributes(FDS, CK)
    return set(R) - set(primes)

: 

In [ ]:
def is2NF(FDS, CK):
    R = getRelations()
    candidates = CK
    primes = getPrimeAttributes(FDS, candidates)
    non_primes = getNonPrimeAttributes(R, FDS, candidates)

    for FD in FDS:
        rhs = FD[1]
        lhs = FD[0]

    for att,nonatt in zip(primes,non_primes):
        if (nonatt in rhs and CK not in lhs) :
            print("\t", lhs, "->", rhs, "breaks 2NF requirements (", lhs, "is NOT Candidate Key )")
            return False

    return True

: 

In [ ]:
def is3NF(FDS, CK):
    '''
    Checks if a relation is in 3NF
    '''
    R = getRelations()
    superkeys = getSuperKeys(R, FDS)
    primes = getPrimeAttributes(FDS, CK)
    non_primes = getNonPrimeAttributes(R, FDS, CK)
    
    #print(f"Super Keys : {superkeys[1]}, {type(superkeys)}")
    for FD in FDS:
        rhs = FD[1]
        lhs = FD[0]
        
        #print(lhs)
        if lhs in superkeys:
            continue

        for patt,natt in zip(primes,non_primes):
            if (natt in rhs and (CK or superkeys) not in lhs) or ( (natt in lhs) and (natt in rhs) ):
                print("\t", lhs, "->", rhs, "breaks 3NF requirements (", natt, "is non-prime )")
                return False

    return True

: 

In [ ]:
def isBCNF(FDS, CK):
    '''
    Checks if a relation is in BCNF
    '''
    R = getRelations()
    superkeys = getSuperKeys(R, FDS)

    for FD in FDS:
        rhs = FD[1]
        lhs = FD[0]
        #print(lhs)
        if ((CK or superkeys) not in lhs):
            print("\t", lhs, "->", rhs, "breaks BCNNF requirements (", lhs, "is non-superkey )")
            return False

    return True

: 

In [ ]:
def exe(FDS, CK):
    print(f"Candidate Key is : {CK}")
    print(f"Functional Dependencies are : {FDS}")
    #verify1NF()
    if(is2NF(FDS, CK)):
        print("Relation is in 2 Normal Form.")
        if(is3NF(FDS, CK)):
            print("Relation is in 3 Normal Form.")
            if(isBCNF(FDS, CK)):
                print("Relation is in BCNF Normal Form.")
    
FDS = MainCode()
CK = fetchCK()[0]    
exe(FDS, CK)

: 